<a href="https://colab.research.google.com/github/tcharos/NLP-Toxicity-Detection/blob/main/AIDL_CS01_NLP_Project_task_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIDL_B_CS01: Advanced NLP Project

## LLM Tuning with DPO (Gordon Ramsay Alignment)

In [10]:
import os
import sys

IN_COLAB = 'google.colab' in sys.modules
BASE_DIR = "/content" if IN_COLAB else "."
TOXICITY_PATH = os.path.join(BASE_DIR, "data_sets/toxicity")

if IN_COLAB:
    print("Running in Google Colab. Installing NLP stack...")
    !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
    !pip install -q -U transformers datasets accelerate peft trl sentence-transformers
else:
    print("Running locally. Checking Mac-specific requirements...")
    !{sys.executable} -m pip install -q "tensorflow==2.16.2" "tensorflow-macos==2.16.2" "tf-keras~=2.16"
    !{sys.executable} -m pip install unsloth-mlx
    !{sys.executable} -m pip install -q -U transformers datasets accelerate peft trl sentence-transformers

os.environ["KERAS_BACKEND"] = "tensorflow"

import torch
import numpy as np
import pandas as pd
import glob
from datasets import Dataset
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import f1_score, confusion_matrix

from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments
)

from peft import LoraConfig, get_peft_model
from trl import DPOTrainer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

if IN_COLAB:
    from unsloth import FastLanguageModel
else:
    from unsloth_mlx import FastLanguageModel

print(f"\nTensorFlow Version: {tf.__version__}")
print("Num GPUs Available (TF): ", len(tf.config.list_physical_devices('GPU')))

HAS_MPS = torch.backends.mps.is_available()
HAS_CUDA = torch.cuda.is_available()

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("PyTorch Device: Mac GPU (Metal)")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("PyTorch Device: Colab GPU (CUDA)")
else:
    device = torch.device("cpu")
    print("PyTorch Device: CPU")

Running locally. Checking Mac-specific requirements...

TensorFlow Version: 2.16.2
Num GPUs Available (TF):  0
PyTorch Device: Mac GPU (Metal)


### Dataset Preparation

In [11]:
# Path to the folder containing colleagues' CSVs
test_folder_path = './data_sets/Ramsay/test'
all_csv_files = glob.glob(os.path.join(test_folder_path, "*.csv"))

valid_dfs = []
required_cols = ["Question", "Polite", "Ramsay"]

for f in all_csv_files:
    try:
        # Try UTF-8 first, fallback to cp1252 if it fails
        try:
            temp_df = pd.read_csv(f, encoding='utf-8')
        except UnicodeDecodeError:
            temp_df = pd.read_csv(f, encoding='cp1252')
        
        # Check if the required columns exist
        if all(col in temp_df.columns for col in required_cols):
            valid_dfs.append(temp_df[required_cols])
        else:
            print(f"Skipping {f}: Missing required columns. Found: {temp_df.columns.tolist()}")
            
    except Exception as e:
        print(f"Could not load {f} due to error: {e}")

# Combine only the valid ones
if valid_dfs:
    all_colleagues_data = pd.concat(valid_dfs, ignore_index=True)
    # Requirement 4: Save to test.csv
    all_colleagues_data.to_csv("test.csv", index=False)
    
    # Take 500 for training
    train_df = all_colleagues_data.sample(n=min(500, len(all_colleagues_data)), random_state=42)
    print(f"Successfully loaded {len(valid_dfs)} files.")
    print(f"Total training rows available: {len(all_colleagues_data)}")
else:
    print("No valid CSV files were loaded!")

Could not load ./data_sets/Ramsay/test/ramsay_mscaidl_0115.csv due to error: Error tokenizing data. C error: Expected 3 fields in line 4, saw 4

Could not load ./data_sets/Ramsay/test/CS02_Task4_msaidl_0089.csv due to error: Error tokenizing data. C error: Expected 4 fields in line 72, saw 5

Skipping ./data_sets/Ramsay/test/RamsayQ_A.csv: Missing required columns. Found: ['AIDL_ID', 'student_question', 'polite_answer', 'ramsay_answer']
Could not load ./data_sets/Ramsay/test/0067_Preference Alignment Dataset.csv due to error: Error tokenizing data. C error: Expected 2 fields in line 3, saw 3

Skipping ./data_sets/Ramsay/test/Preference Alignment Dataset_mscaidl-0122.csv: Missing required columns. Found: ['ID', 'Question', 'Polite Answer', 'Gordon Ramsay Style (Toxic)']
Could not load ./data_sets/Ramsay/test/Toxic Dataset.csv due to error: Error tokenizing data. C error: Expected 1 fields in line 22, saw 2

Skipping ./data_sets/Ramsay/test/gordon_ramsay_prompt_engineering.csv: Missing r

In [12]:
val_folder_path = './data_sets/Ramsay/val'
your_csv_file = glob.glob(os.path.join(val_folder_path, "mscaidl-0077_ramsay_dataset.csv"))[0]

eval_df = pd.read_csv(your_csv_file)
print(eval_df.head())

                      AIDL_ID;Question;Polite;Ramsay
0  0077;"What is the purpose of a Padding token?"...
1  0077;"What is the purpose of Dropout?";"Dropou...
2  0077;"Why normalize input data?";"Normalizatio...
3  0077;"What is a Learning Rate?";"It is a hyper...
4  0077;"Why use Batch Normalization?";"It stabil...


### Functions

### SLM from usloath (not Zephyr)



In [ ]:
# model_name = "unsloth/Llama-3.2-3B-Instruct"
model_name = "unsloth/Llama-3.2-1B-Instruct"

max_seq_length = 2048 # Can handle longer contexts if needed
dtype = None # Auto-detect (Float16 or Bfloat16)
load_in_4bit = True # Essential for DPO memory efficiency

# 2. Load the model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# 3. Add LoRA Adapters
# This allows us to train the model efficiently by only updating a small percentage of weights
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rank
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Optimized for Unsloth
    bias = "none",    # Optimized for Unsloth
    use_gradient_checkpointing = "unsloth", # Reduces VRAM usage
    random_state = 3407,
)

print(f"Model {model_name} loaded successfully with LoRA adapters.")

Fetching 9 files:  33%|███▎      | 3/9 [01:13<02:26, 24.37s/it]
Cancellation requested; stopping current tasks.


KeyboardInterrupt: 

In [ ]:
### 5.2. Model and DPO Trainer Setup

# **ACTION REQUIRED: Choose your base model**
BASE_MODEL = "facebook/opt-125m" # Use a small model for development, or a larger one if resources allow

# 1. Load the base model and tokenizer
# model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, torch_dtype=torch.bfloat16) # Use a suitable dtype
# tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
# tokenizer.pad_token = tokenizer.eos_token # Ensure pad token is set

# 2. Setup PEFT/LoRA configuration
# peft_config = LoraConfig(
#     r=16,
#     lora_alpha=16,
#     target_modules=["q_proj", "v_proj"], # Check model documentation for correct layers
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM",
# )

# 3. DPO Training Arguments
# training_args_dpo = TrainingArguments(
#     output_dir="./dpo_results_ramsay",
#     num_train_epochs=1,
#     per_device_train_batch_size=4,
#     gradient_accumulation_steps=4,
#     logging_steps=10,
#     learning_rate=5e-5,
#     remove_unused_columns=False,
#     save_strategy="epoch",
#     fp16=True, # Use fp16/bf16 if supported
#     report_to="none"
# )

# 4. Initialize DPOTrainer
# dpo_trainer = DPOTrainer(
#     model=model,
#     ref_model=None, # Set to None for implicit reference model loading
#     args=training_args_dpo,
#     beta=0.1,
#     train_dataset=dpo_hf_dataset['train'],
#     eval_dataset=dpo_hf_dataset['test'],
#     tokenizer=tokenizer,
#     peft_config=peft_config,
# )

# 5. Train Placeholder
# print("Starting DPO Training...")
# dpo_trainer.train()

# 6. Save the final model (LoRA weights)
# dpo_trainer.save_model("ramsay_dpo_adapter")

# **ACTION REQUIRED: Inference Test**
# Test the fine-tuned model with a new question to verify the Ramsay persona.
# from peft import PeftModel
# ft_model = PeftModel.from_pretrained(model, "ramsay_dpo_adapter")
# ft_model.eval()
# print("\n--- DPO Fine-Tuned Model Test ---")
# test_prompt = "Why is batch normalization useful?"
# ... generate response ...
